In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2024-04-19


Found rundate 2024-04-18 10:00:03 < 2024-04-19 at commit 5657f24f; returning commit ae9d4bb


'2024-04-19 10:00:03'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2024-04-19 10:00:03, NJSP was reporting 183 YTD deaths
Current YTD Deaths (2025-04-19 10:00:02-04:00): 147
Previous year YTD Deaths (adjusted; 2024-04-19 10:00:03): 181.8236241673209
Projected 2025 total: 622.5053991224202
-19.2% change


FAUQStats cache miss: 2024, Fri Apr 19 10:00:03 EDT 2024


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

CCODE      CNAME MCODE             MNAME HIGHWAY  \
ACCID                                                    
12918    12  Middlesex  1218   Plainsboro Twsp       1   
12923    11     Mercer  1111           Trenton     129   
12932    01   Atlantic  0112     Hamilton Twsp     446   
12933    02     Bergen  0248       Ramsey Boro     NaN   
12927    20      Union  2009       Linden City      27   
...     ...        ...   ...               ...     ...   
13124    04     Camden  0427   Pennsauken Twsp     130   
13126    14     Morris  1431   Pequannock Twsp      23   
13125    07      Essex  0709    Irvington Town     444   
13130    12  Middlesex  1220  South Amboy City     NaN   
13133    04     Camden  0412  Cherry Hill Twsp      41   

                               LOCATION  FATALITIES  FATAL_D  FATAL_P  \
ACCID                                                                   
12918                   State Highway 1         1.0      0.0      0.0   
12923                 State Highway 129         1.0      1.0      0.0   
12932  Atlantic City Expressway MP 20.5         1.0      1.0      0.0   
12933                       East Oak St         1.0      1.0      0.0   
12927                  State Highway 27         1.0      0.0      0.0   
...                                 ...         ...      ...      ...   
13124                 State Highway 130         1.0      0.0      1.0   
13126                  State Highway 23         1.0      1.0      0.0   
13125     Garden State Parkway MP 145.1         1.0      1.0      0.0   
13130                    South Broadway         1.0      0.0      0.0   
13133                                41         1.0      0.0      0.0   

       FATAL_T  FATAL_B          STREET  INJURIES                        dt  
ACCID                                                                        
12918      1.0      0.0             NaN       NaN 2024-01-01 01:56:00-05:00  
12923      0.0      0.0             NaN       NaN 2024-01-03 12:14:00-05:00  
12932      0.0      0.0             NaN       NaN 2024-01-05 09:14:00-05:00  
12933      0.0      0.0     East Oak St       NaN 2024-01-05 14:12:00-05:00  
12927      1.0      0.0             NaN       NaN 2024-01-06 08:37:00-05:00  
...        ...      ...             ...       ...                       ...  
13124      0.0      0.0             NaN       NaN 2024-04-16 15:22:00-04:00  
13126      0.0      0.0             NaN       NaN 2024-04-16 19:42:00-04:00  
13125      0.0      0.0             NaN       NaN 2024-04-17 00:45:00-04:00  
13130      1.0      0.0  South Broadway       NaN 2024-04-17 16:14:00-04:00  
13133      1.0      0.0             NaN       NaN 2024-04-18 21:03:00-04:00  

[175 rows x 14 columns]

Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

driver  passenger  pedestrian  cyclist  crashes
county                                                     
Atlantic         6          0           3        0        9
Bergen           5          1           1        0        7
Burlington       6          4           3        0       12
Camden           9          0           4        0       13
Cape May         2          0           0        0        2
Cumberland       3          2           2        0        6
Essex            4          1           5        1       11
Gloucester       8          2           0        0        8
Hudson           0          0           2        0        2
Hunterdon        2          0           0        0        2
Mercer           3          0           3        0        6
Middlesex        4          1           1        0        6
Monmouth         3          1           0        0        3
Morris           3          2           0        1        3
Ocean            9          3           4        0       13
Passaic          2          2           3        1        8
Salem            2          0           2        0        4
Somerset         5          2           0        0        7
Sussex           0          1           0        0        1
Union            2          2           3        0        7
Warren           2          3           1        0        6

Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

29.7% through the year, 70.3% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

prv_ytd  prv_end  cur_ytd
county     type                              
Atlantic   driver         6       31        6
Bergen     driver         6       14        5
Burlington driver         9       32        6
Camden     driver         5       19        9
Cape May   driver         1        8        2
...                     ...      ...      ...
Salem      crashes        4       14        4
Somerset   crashes        2       18        7
Sussex     crashes        2       12        1
Union      crashes        4       35        7
Warren     crashes        4       14        6

[105 rows x 3 columns]

In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type        crashes  cyclist  driver  passenger  pedestrian
county                                                     
Atlantic         47        1      31          1          16
Bergen           28        1      13          5          11
Burlington       44        1      27          7          11
Camden           36        0      26          1          10
Cape May         12        0      11          0           1
Cumberland       27        0      18          8           8
Essex            49        2      19          7          28
Gloucester       31        0      23          8           4
Hudson           15        0       6          0          11
Hunterdon         9        0       6          1           1
Mercer           27        1      12          2          19
Middlesex        35        1      19          6           9
Monmouth         27        3      12          3          10
Morris           19        2      13          4           4
Ocean            46        1      32          8          11
Passaic          29        2      10          4          22
Salem            14        2       7          1           3
Somerset         35        0      27          3           5
Sussex           10        1       4          3           1
Union            45        1      23          8          14
Warren           17        0      11          6           2

In [9]:
projected.to_csv(PROJECTED_CSV)